# Nova seção

In [26]:
!pip install -q google.adk

In [9]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [10]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [12]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [13]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [14]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [31]:
##########################################
# --- Agente 1: métricas --- #
##########################################
def agente_metricas(genero, idade, peso,altura,nivel_atv_fisica):

    metricas = Agent(
        name="agente_metricas",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente especializado em nutrição, você deve calcular a taxa de metabolismo basal e imc utilizando o genero, idade, peso, altura
        e o nivel de atividade fisica.
        Sua resposta deve ser sucinta contendo somente o resultado dos calculos e exibindo no seguinte formato:

        Taxa de metabolismo basal :
        imc :
        """,
        description="Agente que calcula as métricas",
        tools=[google_search]
    )

    entrada_do_agente_metricas = f"genero: {genero}\nidade: {idade}\npeso: {peso}\naltura: {altura}\nnivel de atividade fisica: {nivel_atv_fisica}"

    metricas = call_agent(metricas, entrada_do_agente_metricas)
    return metricas

In [19]:
################################################
# --- Agente 2: Monta a dieta --- #
################################################
def agente_dieta(nome, genero, idade, peso,altura,metricas,objetivo,nivel_atv_fisica,quantidade_refeicoes,restricoes):
    dieta = Agent(
        name="agente_dieta",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente de dieta #################################################
        instruction="""
        Você é especializado em nutrição e deverá montar uma dieta saúdavel de acordo com o objetivo,nivel de atividade fisica, genero, idade, peso, altura,
        taxa de metabolismo basal, imc, quantidade de refeicoes estipuladas alem de não pode conter alimentos que estejam como restricoes.
        Cada refeição deverá ter 3 opções de combinações de alimentos para ajudar a atingir a quantidade de macro e micro nutrientes.
        A quantidade de cada alimento deve estar especificada em gramas.
        Sua resposta deve ser sucinta e respeitando o formato abaixo:

        Nome :
        Idade :
        Peso :
        Taxa de Metabolismo Basal :
        Imc :
        Objetivo :
        Nivel de atividade fisica :
        Quantidade de refeições diarias :
        Restrições alimentares :

        Dieta
        ...
        """,
        description="Agente que monta a dieta de acordo com as métricas",
        tools=[google_search]
    )

    entrada_do_agente_dieta = f"nome:{nome}\ngenero: {genero}\nidade: {idade}\npeso: {peso}\naltura: {altura}\nmetricas: {metricas}\nobjetivo: {objetivo}\nnivel de atividade fisica: {nivel_atv_fisica}\nquantidade_refeicoes: {quantidade_refeicoes}\nrestrições: {restricoes}"
    # Executa o agente
    saida_dieta = call_agent(dieta, entrada_do_agente_dieta)
    return saida_dieta

In [32]:
print("🚀 Iniciando o Sistema auxiliar para nutricionistas 🚀")

# --- Obter o nome do paciente ---
nome = input("❓ Por favor, Qual o nome do paciente? ")
while(not nome):
    nome = input("❓ Por favor, digite o nome do paciente ")

# --- Obter a idade do paciente ---
idade = input("❓ Qual a idade do paciente?")
while(not idade):
    idade = input("❓ Por favor, digite a idade do paciente ")

#---Verifica se é numero-----#
while True:
    if idade.isdigit():
        break  # Sai do loop se a entrada for válida
    else:
        print("Entrada inválida. Digite apenas números.")
        idade = input("❓ Por favor, digite a idade do paciente ")

# --- Obter o gênero do paciente ---
genero = input(f"❓Qual o genero biológico? (Masculino ou feminino) ")
while(not genero):
    genero = input("❓ Por favor, qual o genero biológico do paciente?(Masculino ou feminino) ")

# --- Obter o peso do paciente ---
peso = input("❓ Qual o peso? ")
while(not peso):
    peso = input("❓ Por favor, qual o peso do paciente? ")

#---Verifica se é numero-----#
while True:
    if peso.isdigit():
        break  # Sai do loop se a entrada for válida
    else:
        print("Entrada inválida. Digite apenas números.")
        peso = input("❓ Por favor, qual o peso do paciente? ")

# --- Obter o altura do paciente ---
altura = input("❓ Altura? (exemplo: 1.73) ")
while(not altura):
    altura = input("❓ Por favor, a altura do paciente? (exemplo: 1.73) ")

# --- Obter o objetivo do paciente ---
objetivo = input("❓ Qual o objetivo do paciente?(Hipertrofria,emagrecimento,competição,manutenção) ")
while(not altura):
    altura = input("❓ Por favor, digite o objetivo?(Hipertrofria,emagrecimento,competição,manutenção) ")

# --- Obter o altura do paciente ---
exercicio = input("❓ O paciente pratica exercicios quantas vezes por semana? (0 - 7) ")
while(not altura):
    exercicio = input("❓ Por favor, o paciente pratica exercicios quantas vezes por semana? (0 - 7) ")

#---Verifica se é numero-----#
while True:
    if exercicio.isdigit():
        break  # Sai do loop se a entrada for válida
    else:
        print("Entrada inválida. Digite apenas números.")
        exercicio = input("❓ Por favor, o paciente pratica exercicios quantas vezes por semana? (0 - 7) ")

#------Nivel de atividade fisica-----------#
if int(exercicio)==0 :
    nivel_atv_fisica="Sedentário"
elif int(exercicio)>=1 and int(exercicio)<=2 :
    nivel_atv_fisica="Levemente ativo"
elif int(exercicio)>=3 and int(exercicio)<=5 :
    nivel_atv_fisica="Moderadamente ativo"
elif int(exercicio)>5:
    nivel_atv_fisica="Muito ativo"
else:
    nivel_atv_fisica="Não informado"

# --- Obter quantidade de refeições ---
quantidade_refeicoes = input("❓ Quantas refeições diarias são mais adequadas para o paciente? ")
while(not quantidade_refeicoes):
    quantidade_refeicoes = input("❓ Por favor,qual a quantidade refeições diarias mais adequada para o paciente? ")

# --- Obter as restrições do paciente---
restricoes_sim_nao = input("❓ O paciente possui alguma alergia  ou restrições de alimentos? (s ou n) ")
while(not restricoes_sim_nao):
    restricoes_sim_nao = input("❓ Por favor,o paciente possui alguma restrição de alimentos? (s ou n) ")

if restricoes_sim_nao=='s' or restricoes_sim_nao=='S' :
    restricoes= input("❓ Por favor,Digite as restrições. Exemplo(amendoim,leite,etc..) ")
else:
    print("Ok! O paciente não possui restrições" )
    restricoes="Sem restrições de alimentos "


# Inserir lógica do sistema de agentes ################################################

metricas = agente_metricas(genero, idade, peso,altura,nivel_atv_fisica)
print("\n--- 📝 Resultado do Agente 1 métricas ---\n")
display(to_markdown(metricas))
print("--------------------------------------------------------------")

dieta = agente_dieta(nome, genero, idade, peso,altura,metricas,objetivo,nivel_atv_fisica,quantidade_refeicoes,restricoes)
print("\n--- 📝 Resultado do Agente 2 (Dieta) ---\n")
display(to_markdown(dieta))
print("-------------------Dieta Gerada!!!---------------")



🚀 Iniciando o Sistema auxiliar para nutricionistas 🚀
❓ Por favor, Qual o nome do paciente? Rodrigo
❓ Qual a idade do paciente?34
❓Qual o genero biológico? (Masculino ou feminino) Masculino
❓ Qual o peso? 105
❓ Altura? (exemplo: 1.73) 1.73
❓ Qual o objetivo do paciente?(Hipertrofria,emagrecimento,competição,manutenção) emagrecimento
❓ O paciente pratica exercicios quantas vezes por semana? (0 - 7) 4
❓ Quantas refeições diarias são mais adequadas para o paciente? 6
❓ O paciente possui alguma alergia  ou restrições de alimentos? (s ou n) s
❓ Por favor,Digite as restrições. Exemplo(amendoim,leite,etc..) leite

--- 📝 Resultado do Agente 1 métricas ---



> Taxa de metabolismo basal: 2041.15
> imc: 35.05
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Dieta) ---



> Okay, Rodrigo. Com os seus dados e restrições, posso montar uma dieta para emagrecimento.
> 
> 
> Com prazer, aqui está uma sugestão de dieta com 6 refeições diárias, focada no emagrecimento, considerando seu nível de atividade física, restrição ao leite e suas métricas:
> 
> **Nome:** Rodrigo
> **Gênero:** Masculino
> **Idade:** 34
> **Peso:** 105 kg
> **Altura:** 1.73 m
> **Taxa de Metabolismo Basal:** 2041.15 kcal
> **IMC:** 35.05
> **Objetivo:** Emagrecimento
> **Nível de atividade física:** Moderadamente ativo
> **Quantidade de refeições diárias:** 6
> **Restrições alimentares:** Leite
> 
> **Dieta**
> 
> **Café da Manhã (Aproximadamente 300-350 kcal)**
> 
> *   **Opção 1:**
>     *   3 ovos mexidos (claras e gemas) (150g)
>     *   1 fatia de pão integral (30g)
>     *   1/2 abacate (40g)
> *   **Opção 2:**
>     *   Smoothie de 1 scoop de whey protein isolado (sem lactose), 200ml de água de coco, 1/2 banana (40g), 1 colher de sopa de chia
>     *   2 biscoitos de arroz integral.
> *   **Opção 3:**
>     *   Crepioca feita com 2 ovos (100g) e 2 colheres de sopa de tapioca (40g) recheada com frango desfiado (50g).
> 
> **Lanche da Manhã (Aproximadamente 150-200 kcal)**
> 
> *   **Opção 1:**
>     *   1 maçã com casca (150g) e 10 castanhas de caju.
> *   **Opção 2:**
>     *   Iogurte de coco sem açúcar (150g) com 1 colher de sopa de granola sem açúcar.
> *   **Opção 3:**
>     *   1 pera (150g) com 2 colheres de sopa de pasta de amendoim integral.
> 
> **Almoço (Aproximadamente 400-450 kcal)**
> 
> *   **Opção 1:**
>     *   150g de peito de frango grelhado
>     *   1 xícara de arroz integral (100g)
>     *   1 xícara de brócolis cozido no vapor (100g)
>     *   Salada de folhas verdes à vontade (alface, rúcula, espinafre) temperada com azeite e limão.
> *   **Opção 2:**
>     *   150g de salmão assado
>     *   1 batata doce média (150g)
>     *   Salada de pepino e tomate com azeite e vinagre.
> *   **Opção 3:**
>     *   150g de carne magra grelhada (patinho, alcatra)
>     *   1 xícara de quinoa cozida (100g)
>     *   1 xícara de abobrinha refogada.
> 
> **Lanche da Tarde (Aproximadamente 150-200 kcal)**
> 
> *   **Opção 1:**
>     *   1 scoop de whey protein isolado (sem lactose) com água (200ml) e 1 colher de sopa de semente de chia.
> *   **Opção 2:**
>     *   2 ovos cozidos (100g) com uma pitada de sal e pimenta.
> *   **Opção 3:**
>     *   1/2 xícara de mix de nuts e castanhas (30g).
> 
> **Jantar (Aproximadamente 350-400 kcal)**
> 
> *   **Opção 1:**
>     *   Sopa de legumes com frango desfiado (200g legumes + 80g frango)
>     *   1 colher de sopa de azeite extra virgem.
> *   **Opção 2:**
>     *   Salada grande com 150g de atum em água, folhas verdes, tomate, pepino, azeitonas, palmito, azeite e limão.
> *   **Opção 3:**
>     *   Omelete com 3 claras e 1 gema (120g), espinafre e tomate picado.
> 
> **Ceia (Aproximadamente 100-150 kcal)**
> 
> *   **Opção 1:**
>     *   1 iogurte de coco sem açúcar (170g).
> *   **Opção 2:**
>     *   1 xícara de chá de camomila sem açúcar
>     *   30g de amêndoas.
> *   **Opção 3:**
>     *   1 fruta pequena (ex: 1/2 mamão papaia).
> 
> **Observações Importantes:**
> 
> *   **Água:** Beba bastante água ao longo do dia (pelo menos 2-3 litros).
> *   **Temperos:** Use temperos naturais como ervas, alho, cebola, pimenta, limão e azeite para dar sabor aos alimentos. Evite temperos industrializados ricos em sódio.
> *   **Variações:** Sinta-se à vontade para variar os legumes e verduras nas saladas e preparações, de acordo com sua preferência e disponibilidade.
> *   **Acompanhamento:** Consulte um nutricionista para um plano alimentar individualizado e adaptado às suas necessidades específicas.
> *   **Atividade física:** Mantenha seu nível de atividade física moderada para auxiliar no emagrecimento e na melhora da saúde geral.
> 
> Lembre-se que esta é apenas uma sugestão. É fundamental consultar um profissional de nutrição para ajustar a dieta às suas necessidades individuais e garantir que ela seja equilibrada e segura para você.


-------------------Dieta Gerada!!!---------------
